In [1]:
!pip install google-cloud-storage

In [2]:
!pip install transformers==4.51.3 datasets peft accelerate bitsandbytes wandb deepspeed lion-pytorch

In [3]:
import transformers
print(transformers.__version__)

4.51.3


In [4]:
!pip install --upgrade --pre triton

# Import Data

In [5]:
import json
from google.cloud import storage
import os

def save_json(data, filename):
    # Get the directory from the filename
    directory = os.path.dirname(filename)

    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the data to the file
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def list_files_in_bucket(bucket_name, prefix=""):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    # Print the list of file names in the bucket
    print("Files in the bucket:")
    for blob in blobs:
        print(blob.name)

def load_json_from_gcs(bucket_name, file_name):
    from google.cloud import storage
    import json

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    if not file_name.endswith('.jsonl'):  # Ensure it's a JSONL file
        raise ValueError(f"The specified file '{file_name}' is not a JSONL file.")

    concatenated_data = []  # To accumulate JSON objects
    try:
        # Download and decode the file content
        content = blob.download_as_string().decode('utf-8')
        # Split content by lines and load each line as a separate JSON object
        for line in content.splitlines():
            if line.strip():  # Only parse non-empty lines
                json_obj = json.loads(line)
                concatenated_data.append(json_obj)  # Add JSON object to the list
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in {file_name}: {e}")

    # Return the JSON as a string for output
    return json.dumps(concatenated_data, indent=4)  # Prettify the JSON output

def load_csv_from_gcs(bucket_name, file_name):
    from google.cloud import storage
    import pandas as pd
    from io import StringIO  # Corrected import for StringIO

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    if not file_name.endswith('.csv'):  # Ensure it's a CSV file
        raise ValueError(f"The specified file '{file_name}' is not a CSV file.")

    try:
        # Download CSV content and load it into a pandas DataFrame
        content = blob.download_as_string().decode('utf-8')
        data = pd.read_csv(StringIO(content))  # Use StringIO to parse the CSV content
    except Exception as e:
        print(f"Error loading CSV file '{file_name}': {e}")
        return None

    return data


def save_csv_to_gcs(bucket_name, file_name, dataframe):
    from google.cloud import storage
    import pandas as pd

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    if not file_name.endswith('.csv'):
        raise ValueError(f"The specified file '{file_name}' is not a CSV file.")

    try:
        # Convert the DataFrame to CSV and upload it to GCS
        csv_content = dataframe.to_csv(index=False)  # Convert DataFrame to CSV string
        blob.upload_from_string(csv_content, content_type='text/csv')
        print(f"File '{file_name}' successfully saved to bucket '{bucket_name}'.")
    except Exception as e:
        print(f"Error saving CSV file '{file_name}': {e}")

In [6]:
train_data = load_csv_from_gcs("mddi-reach-conversation", "modernbert/modernbert_train_truncated.csv")
augmented_full_df = load_csv_from_gcs("mddi-reach-conversation", "modernbert/modernbert_augmented_truncated.csv")
test_data = load_csv_from_gcs("mddi-reach-conversation", "modernbert/modernbert_test_truncated.csv").rename({'key_point': 'stance', 'person_id': 'user'}, axis=1)
vali_data = load_csv_from_gcs("mddi-reach-conversation", "modernbert/modernbert_val_truncated.csv").rename({'key_point': 'stance', 'person_id': 'user'}, axis=1)
print(augmented_full_df.shape)
augmented_full_df.tail()

(4943, 7)


,chat_group_id,contributor,stance,content_concat,label,human_label,prompt
4938,1279,333,Contributors raised strong concerns regarding ...,"Contributor955: Not to forget, the dynamics of...",0,neutral,<s>[INST]Determine whether Contributor333 hold...
4939,457,728,Contributors expressed concern over the govern...,Contributor775: While affordable housing is cr...,0,neutral,<s>[INST]Determine whether Contributor728 hold...
4940,757,130,Contributors have expressed concern over the r...,Contributor508: Perhaps we could consider that...,1,agree,<s>[INST]Determine whether Contributor130 hold...
4941,637,708,Contributors expressed concern over the rising...,Contributor556: Access to knowledge is the beg...,1,agree,<s>[INST]Determine whether Contributor708 hold...
4942,1928,619,Contributors highlighted the importance of inc...,Contributor288: What about the pressure it mig...,0,disagree,<s>[INST]Determine whether Contributor619 hold...


In [7]:
train_data.columns, test_data.columns, augmented_full_df.columns

(Index(['group', 'user', 'stance', 'topic_group', 'human_label', 'agreement',
        'group_user', 'label', 'id', 'content_concat'],
       dtype='object'),
 Index(['discussion_id', 'stance', 'contributor_name', 'key_point_id',
        'contributor_id', 'expressed_view', 'user', 'topic', 'topic_statement',
        'label', 'content_concat', 'prompt', 'translated_content'],
       dtype='object'),
 Index(['chat_group_id', 'contributor', 'stance', 'content_concat', 'label',
        'human_label', 'prompt'],
       dtype='object'))

In [8]:
import pandas as pd
train_df = pd.concat([
    train_data[[ 'label', 'stance', 'content_concat', 'human_label', 'topic_group', 'agreement', 'user', 'group_user']],
    augmented_full_df[[  'stance', 'content_concat', 'label']],
    ]
).reset_index(drop=True)
print(train_df.shape)
train_df.head(2)

(7348, 8)


,label,stance,content_concat,human_label,topic_group,agreement,user,group_user
0,0,Contributor shared different perspectives on p...,Contributor926: Huh! They spoil market!\nDo yo...,no opinion,national_day,unanimous,04195570-f8b3-4eab-866f-32808d77d8e1,104195570-f8b3-4eab-866f-32808d77d8e1
1,0,Contributor expressed differing levels of trus...,Contributor926: Huh! They spoil market!\nDo yo...,disagree,national_day,majority,04195570-f8b3-4eab-866f-32808d77d8e1,104195570-f8b3-4eab-866f-32808d77d8e1


# Training Code

## Tokenizer

In [10]:
import torch
model_name = "Jsevisal/CrossEncoder-ModernBERT-base-qnli"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer


def tokenize_dataset(df, tokenizer, device, max_length=512):
    """
    Tokenizes the dataset using the specified tokenizer.

    Args:
        dataset (Dataset): The Hugging Face dataset to be tokenized.
        tokenizer (AutoTokenizer): Tokenizer from Hugging Face Transformers.
        max_length (int): Maximum sequence length for padding/truncation.

    Returns:
        Dataset: Tokenized Hugging Face dataset.
    """
    # Define the tokenization function
    def tokenize_function(examples, max_length, device):
        inputs = tokenizer(
            examples,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",

        )
        return {key: value.to(device) for key, value in inputs.items()}

    df['sentence1'] = df['stance'].apply(lambda x: str(x))
    df['sentence2'] = df['content_concat'].apply(lambda x: str(x))

    inputs = df[["sentence1", 'sentence2']].values.tolist()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenized = tokenize_function(inputs, max_length, device)

    # Create a new dataset from the tokenized data
    tokenized_dataset = Dataset.from_dict(tokenized)
    tokenized_dataset = tokenized_dataset.add_column('label', df['label'])
    return tokenized_dataset

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataloader = tokenize_dataset(train_df, tokenizer, device, max_length=8192)
test_dataloader = tokenize_dataset(test_data, tokenizer, device, max_length=8192)
vali_dataloader = tokenize_dataset(vali_data, tokenizer, device, max_length=8192)

## Training

In [8]:
import torch
import torch.nn as nn
from lion_pytorch import Lion
import wandb
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score,
    auc,
    accuracy_score
)
from datetime import datetime
from sklearn.preprocessing import LabelBinarizer, label_binarize
from tqdm import tqdm

from typing import Callable
from torch.optim import Optimizer
from sentence_transformers.evaluation.SentenceEvaluator import SentenceEvaluator
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch import nn
from tqdm.autonotebook import tqdm, trange
import torch
from sentence_transformers.SentenceTransformer import SentenceTransformer
import numpy as np
import logging
import torch.nn.functional as F
import time
from transformers import get_scheduler
from collections import Counter
from accelerate import Accelerator




[2025-05-11 06:17:21,700] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [9]:

class CrossEncoderForMultiClass():
    def __init__(
        self,
        model_name: str,
        num_labels: int = None,
        batch_size: int = 32,
        output_path: str = None
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if num_labels == 1:
          ignore_mismatched_sizes = True
        else:
          ignore_mismatched_sizes = False
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels = num_labels,
            ignore_mismatched_sizes=ignore_mismatched_sizes,
        ).to(device)
        self.num_labels= num_labels
        print("Model loaded successfully")

        self.model.to(self.device)
        self.batch_size = batch_size
        self.output_path = output_path
        self.early_stopping = 0
        # Create the output path if it doesn't exist
        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)

    def create_batch(self, dataset, batch_size=None, shuffle=True):
        if batch_size is None:
            batch_size = batch_size

        # If shuffle is True, shuffle the dataset first
        if shuffle:
            dataset = dataset.shuffle(seed=42)  # Shuffle with a seed for reproducibility

        # Set the format for Hugging Face datasets (before passing to DataLoader)
        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

        dataloader = DataLoader(dataset, batch_size=batch_size)

        return dataloader

    def fit(
        self,
        train_data: DataLoader,
        evaluator: SentenceEvaluator = None,
        epochs: int = 1,
        loss_fct=None,
        #activation_fct = nn.Identity(),
        scheduler: str = "WarmupLinear",
        warmup_steps: int = 10000,
        optimizer_class: type[Optimizer] = torch.optim.AdamW,
        optimizer_params: dict[str, object] = {"lr": 2e-5},
        weight_decay: float = 0.01,
        evaluation_steps: int = 0,
        save_best_model: bool = True,
        #max_grad_norm: float = 1,
        #use_amp: bool = False,
        callback: Callable[[float, int, int], None] = None,
        show_progress_bar: bool = True,
    ):

        # Initialize variables
        self.best_score = -9999999
        num_train_steps = int(len(train_data) * epochs)


        # Define the loss function
        if loss_fct is None:
            # Initialize counters
            #N_pos = 0
            #N_neg = 0

            # Iterate through the dataloader
            #for batch in train_dataloader:
            #    targets = batch["label"]
            #    # Assuming targets are binary (0 or 1)
            #    N_pos += sum(targets == 1)
            #    N_neg += sum(targets == 0)
            #pos_weight = torch.tensor([N_neg / N_pos])

            loss_fct = nn.BCEWithLogitsLoss() if self.num_labels == 1 else nn.CrossEntropyLoss()

        # Prepare optimizer and scheduler
        param_optimizer = list(self.model.named_parameters())

        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
            },
            {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]

        optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

        if isinstance(scheduler, str):
            scheduler_optimizer = SentenceTransformer._get_scheduler(
                optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps
            )
        print("Starting model training...")
        training_steps = 0
        gradient_accumulation_steps = self.batch_size
        micro_batch_size = 1

        self.early_stopping = 0
        for epoch in trange(epochs, desc="Epoch", disable=not show_progress_bar):
            train_dataloader = self.create_batch(train_data, batch_size=micro_batch_size)

            self.model.zero_grad()
            self.model.train()
            loss_values = []
            accumulation_counter = 0

            for batch in tqdm(train_dataloader, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):
                batch_start_time = time.time()

                features = {k: torch.Tensor(v).to(self.device) for k, v in batch.items() if k != 'label'}
                labels = batch['label'].to(self.device)

                self.model.train()
                model_predictions = self.model(**features, return_dict=True)

                if self.num_labels == 1:
                    logits = model_predictions.logits.view(-1).float()
                    labels = labels.float()

                loss_value = loss_fct(logits, labels)
                loss_value = loss_value / gradient_accumulation_steps  # Normalize loss
                loss_value.backward()
                accumulation_counter += 1

                if accumulation_counter % gradient_accumulation_steps == 0:
                    # Optional: clip gradients here
                    # torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
                    optimizer.step()
                    scheduler_optimizer.step()
                    optimizer.zero_grad()
                    accumulation_counter = 0  # reset
                training_steps += 1

                loss_values.append(loss_value.item() * gradient_accumulation_steps)  # rescale to original loss
                batch_end_time = time.time()

                if evaluator is not None and evaluation_steps > 0 and training_steps % evaluation_steps == 0 and training_steps!=0:
                    self.model.eval()
                    self._eval_during_training(
                        evaluator, sum(loss_values)/len(loss_values), batch_end_time-batch_start_time, save_best_model, epoch, training_steps
                    )
                    self.model.zero_grad()
                    self.model.train()
                    if self.early_stopping >= 8:
                        break

    def predict(
        self,
        dataset,
        batch_size: int = 32,
        show_progress_bar = None,
        #num_workers: int = 0,
        #activation_fct = None,
        #apply_softmax = False,
        convert_to_numpy: bool = True,
        convert_to_tensor: bool = False,
        output_prob = True
    ):
        """
        Performs predictions with the CrossEncoder on the given sentence pairs.

        Args:
            sentences (Union[List[Tuple[str, str]], Tuple[str, str]]): A list of sentence pairs [(Sent1, Sent2), (Sent3, Sent4)]
                or one sentence pair (Sent1, Sent2).
            batch_size (int, optional): Batch size for encoding. Defaults to 32.
            show_progress_bar (bool, optional): Output progress bar. Defaults to None.
            num_workers (int, optional): Number of workers for tokenization. Defaults to 0.
            activation_fct (callable, optional): Activation function applied on the logits output of the CrossEncoder.
                If None, nn.Sigmoid() will be used if num_labels=1, else nn.Identity. Defaults to None.
            convert_to_numpy (bool, optional): Convert the output to a numpy matrix. Defaults to True.
            apply_softmax (bool, optional): If there are more than 2 dimensions and apply_softmax=True,
                applies softmax on the logits output. Defaults to False.
            convert_to_tensor (bool, optional): Convert the output to a tensor. Defaults to False.

        Returns:
            Union[List[float], np.ndarray, torch.Tensor]: Predictions for the passed sentence pairs.
            The return type depends on the `convert_to_numpy` and `convert_to_tensor` parameters.
            If `convert_to_tensor` is True, the output will be a torch.Tensor.
            If `convert_to_numpy` is True, the output will be a numpy.ndarray.
            Otherwise, the output will be a list of float values.

        Examples:
            ::

                from sentence_transformers import CrossEncoder

                model = CrossEncoder("cross-encoder/stsb-roberta-base")
                sentences = [["I love cats", "Cats are amazing"], ["I prefer dogs", "Dogs are loyal"]]
                model.predict(sentences)
                # => array([0.6912767, 0.4303499], dtype=float32)
        """
        dataset = self.create_batch(dataset, batch_size = batch_size, shuffle=False)

        pred_scores = []
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
           for batch in tqdm(
                dataset, desc="Iteration", smoothing=0.05, disable=not show_progress_bar
            ):
                features = {k: torch.Tensor(v).to(self.device) for k, v in batch.items() if k != 'label'}
                model_predictions = self.model(**features, return_dict=True)
                #logits = activation_fct(model_predictions.logits)

                #if apply_softmax and len(logits[0]) > 1:
                if output_prob:
                    if self.num_labels == 1:
                        logits = torch.sigmoid(model_predictions.logits)
                    else:
                        logits = F.softmax(model_predictions.logits, dim=1)
                else:
                    logits = model_predictions.logits
                pred_scores.extend(logits)

        if self.num_labels == 1:
            pred_scores = [score[0].cpu() for score in pred_scores]
            if convert_to_tensor:
              pred_scores = torch.Tensor(pred_scores)
            elif convert_to_numpy:
              pred_scores = np.array(pred_scores)
        else:
            if convert_to_tensor:
                pred_scores = torch.stack(pred_scores)
            elif convert_to_numpy:
                pred_scores = np.asarray([score.cpu().detach().float().numpy() for score in pred_scores])

        return pred_scores


    def _eval_during_training(self, evaluator, train_loss, time, save_best_model, epoch, steps) -> None:
        """Runs evaluation during the training"""
        if evaluator is not None:
            score = evaluator(self, train_loss, time, epoch=epoch, steps=steps)
            #if callback is not None:
            #    callback(score, epoch, steps)
            saved_model = False
            if score["vali_f1"] > self.best_score:
                self.best_score = score["vali_f1"]
                self.early_stopping=0
                if save_best_model:
                    self.save(self.output_path)
                    #saved_model = True
            else:
                self.early_stopping+=1
            if save_best_model:
                if saved_model == False:
                    self.save(self.output_path)
            return self.best_score

    def save(self, path: str, *, safe_serialization: bool = True, **kwargs) -> None:
        """
        Saves the model and tokenizer to path; identical to `save_pretrained`
        """

        print(f"Save model to {path}")
        self.model.save_pretrained(path, safe_serialization=safe_serialization, **kwargs)
        self.tokenizer.save_pretrained(path, **kwargs)



In [10]:

class CrossEncoderTrainer():
    def __init__(self,
                  project_name,
                  model_name,
                  num_labels,
                  epochs,
                  lr,
                  train_batch_size,
                  eval_batch_size,
                  warmup_steps,
                  model_save_path,
                  metrics_save_path,
                  weight_decay = 0,
                  evaluation_steps = 100,
                  optimizer_class = "adam",
                  scheduler = None,
                  version = "v1"
              ):
        self.num_labels = num_labels
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = CrossEncoderForMultiClass(
              model_name = model_name,
              num_labels = num_labels,
              batch_size = train_batch_size,
              output_path = model_save_path
        )
        self.model_name = model_name
        self.loss = nn.BCEWithLogitsLoss() if num_labels == 1 else nn.CrossEntropyLoss()
        self.epochs = epochs
        self.warmup_steps = warmup_steps
        self.optimizer_params  = {"lr": lr}
        self.eval_batch_size = eval_batch_size
        self.evaluation_steps = evaluation_steps
        self.weight_decay = weight_decay
        if optimizer_class == "adam":
              self.optimizer_class = torch.optim.AdamW
        elif optimizer_class == "lion":
              self.optimizer_class = Lion
        self.model_save_path = model_save_path
        # Initialize WandB
        config = {
              "model_name": model_name,
              "learning_rate": lr,
              "epochs": epochs,
              "warmup_steps": warmup_steps,
              "evaluation_steps": evaluation_steps,
              "weight_decay": weight_decay,
              "optimizer_class": optimizer_class,
              "batch_size": train_batch_size,
        }
        run_name = self.model_name +  datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        metrics_save_path = (metrics_save_path + "/") if (metrics_save_path[-1] != "/") else metrics_save_path
        self.json_file = metrics_save_path + ("classification_evaluation" + ("_" + model_name if model_name else "") + "_" + version + "_results.json").replace("/", "_")

        wandb.init(
              project=project_name,
              name= run_name[:255],
              config= config
        )
        self.scheduler = scheduler

    def train(self,
              train_dataloader,
              eval_dataloader,
              vali_data
                        ):
        evaluator = CrossEncoderEvaluator(
              eval_dataloader,
              vali_data,
              name = self.model_name,
              batch_size = self.eval_batch_size,
              show_progress_bar= True,
              metrics_save_path=self.json_file,
              loss_fn= self.loss
        )
        print("Training model")
        self.model.fit(
              train_data=train_dataloader,
              evaluator=evaluator,
              evaluation_steps =self.evaluation_steps,
              optimizer_params= self.optimizer_params,
              loss_fct  = self.loss,
              epochs=self.epochs,
              warmup_steps=self.warmup_steps,
              save_best_model = True,
              #callback = self.callback,
              show_progress_bar = True,
              #scheduler = self.scheduler
        )

In [11]:
'''from transformers import get_scheduler, Trainer, TrainingArguments
import torch
import time
from tqdm import tqdm

class CrossEncoderForMultiClass():
    def __init__(
        self,
        model_name: str,
        num_labels: int = None,
        batch_size: int = 32,
        output_path: str = None
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if num_labels == 1:
            ignore_mismatched_sizes = True
        else:
            ignore_mismatched_sizes = False
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=ignore_mismatched_sizes,
        ).to(self.device)

        self.num_labels = num_labels
        self.batch_size = batch_size
        self.output_path = output_path

        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)

    def create_batch(self, dataset, batch_size=None, shuffle=True):
        if batch_size is None:
            batch_size = self.batch_size
        if shuffle:
            dataset = dataset.shuffle(seed=42)
        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
        dataloader = DataLoader(dataset, batch_size=batch_size)
        return dataloader

    def fit(
        self,
        train_data: DataLoader,
        evaluator: SentenceEvaluator = None,
        epochs: int = 1,
        loss_fct=None,
        scheduler_type: str = "linear",  # Added parameter for scheduler type
        warmup_steps: int = 10000,
        optimizer_class: type[Optimizer] = torch.optim.AdamW,
        optimizer_params: dict[str, object] = {"lr": 2e-5},
        weight_decay: float = 0.01,
        evaluation_steps: int = 0,
        save_best_model: bool = True,
        max_grad_norm: float = 1,
        callback: Callable[[float, int, int], None] = None,
        show_progress_bar: bool = True,
    ):
        # Initialize variables
        self.best_score = -9999999
        num_train_steps = int(len(train_data) * epochs)

        # Define loss function
        if loss_fct is None:
            loss_fct = nn.CrossEntropyLoss() if self.num_labels > 1 else nn.BCEWithLogitsLoss()

        # Prepare optimizer and scheduler
        param_optimizer = list(self.model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
            },
            {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
        optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

        # Initialize scheduler using Hugging Face's get_scheduler
        scheduler = get_scheduler(
            name=scheduler_type,  # Scheduler type: "linear", "cosine", etc.
            optimizer=optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=num_train_steps
        )

        # Define training arguments
        training_steps = 0
        gradient_accumulation_steps = self.batch_size  # effective batch size
        micro_batch_size = 2

        for epoch in trange(epochs, desc="Epoch", disable=not show_progress_bar):
            train_dataloader = self.create_batch(train_data, batch_size=micro_batch_size)
            self.model.zero_grad()
            self.model.train()
            loss_values = []
            accumulation_counter = 0

            for batch in tqdm(train_dataloader, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):
                batch_start_time = time.time()

                features = {k: torch.Tensor(v).to(self.device) for k, v in batch.items() if k != 'label'}
                labels = batch['label'].to(self.device)

                model_predictions = self.model(**features, return_dict=True)

                if self.num_labels == 1:
                    logits = model_predictions.logits.view(-1).float()
                    labels = labels.float()

                loss_value = loss_fct(logits, labels)
                loss_value = loss_value / gradient_accumulation_steps  # Normalize loss
                loss_value.backward()
                accumulation_counter += 1

                if accumulation_counter % gradient_accumulation_steps == 0:
                    optimizer.step()
                    scheduler.step()  # Step the scheduler
                    optimizer.zero_grad()
                    accumulation_counter = 0  # reset
                    training_steps += 1

                loss_values.append(loss_value.item() * gradient_accumulation_steps)  # rescale to original loss
                batch_end_time = time.time()

                if evaluator is not None and evaluation_steps > 0 and training_steps % evaluation_steps == 0 and training_steps != 0:
                    self.model.eval()
                    self._eval_during_training(
                        evaluator, sum(loss_values) / len(loss_values), batch_end_time - batch_start_time, save_best_model, epoch, training_steps
                    )

                    self.model.zero_grad()
                    self.model.train()

    def _eval_during_training(self, evaluator, train_loss, time, save_best_model, epoch, steps) -> None:
        """Runs evaluation during the training"""
        if evaluator is not None:
            score = evaluator(self, train_loss, time, epoch=epoch, steps=steps)
            saved_model = False
            if score["vali_f1"] > self.best_score:
                self.best_score = score["vali_f1"]
                if save_best_model:
                    self.save(self.output_path)

            if save_best_model:
                if not saved_model:
                    self.save(self.output_path)

    def save(self, path: str, *, safe_serialization: bool = True, **kwargs) -> None:
        """
        Saves the model and tokenizer to path.
        """
        print(f"Saving model to {path}")
        self.model.save_pretrained(path, safe_serialization=safe_serialization, **kwargs)
        self.tokenizer.save_pretrained(path, **kwargs)
'''

'from transformers import get_scheduler, Trainer, TrainingArguments\nimport torch\nimport time\nfrom tqdm import tqdm\n\nclass CrossEncoderForMultiClass():\n    def __init__(\n        self,\n        model_name: str,\n        num_labels: int = None,\n        batch_size: int = 32,\n        output_path: str = None\n    ):\n        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n        self.tokenizer = AutoTokenizer.from_pretrained(model_name)\n        if num_labels == 1:\n            ignore_mismatched_sizes = True\n        else:\n            ignore_mismatched_sizes = False\n        self.model = AutoModelForSequenceClassification.from_pretrained(\n            model_name,\n            num_labels=num_labels,\n            ignore_mismatched_sizes=ignore_mismatched_sizes,\n        ).to(self.device)\n\n        self.num_labels = num_labels\n        self.batch_size = batch_size\n        self.output_path = output_path\n\n        if output_path is not None:\n            

In [12]:
'''
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from accelerate import Accelerator
import torch
import os
import time
from tqdm import tqdm, trange
from typing import Callable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

class CrossEncoderForMultiClass:
    def __init__(
        self,
        model_name: str,
        num_labels: int = None,
        batch_size: int = 32,
        output_path: str = None
    ):
        # Initialize the accelerator
        self.accelerator = Accelerator()

        # Initialize the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        ignore_mismatched_sizes = num_labels == 1

        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=ignore_mismatched_sizes
        )
        self.num_labels = num_labels
        self.batch_size = batch_size
        self.output_path = output_path

        if output_path:
            os.makedirs(output_path, exist_ok=True)

        self.accelerator.print("Model loaded successfully")

    def create_batch(self, dataset, batch_size=None, shuffle=True):
        if batch_size is None:
            batch_size = self.batch_size

        if shuffle:
            dataset = dataset.shuffle(seed=42)

        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
        return DataLoader(dataset, batch_size=batch_size)

    def fit(
        self,
        train_data,
        evaluator: Callable = None,
        epochs: int = 1,
        loss_fct=None,
        scheduler: str = "WarmupLinear",
        warmup_steps: int = 10000,
        optimizer_class: type = torch.optim.AdamW,
        optimizer_params: dict = {"lr": 2e-5},
        weight_decay: float = 0.01,
        evaluation_steps: int = 0,
        save_best_model: bool = True,
        max_grad_norm: float = 1,
        callback: Callable[[float, int, int], None] = None,
        show_progress_bar: bool = True,
    ):
        # Define training arguments and setup scheduler
        training_args = TrainingArguments(
            output_dir=self.output_path,
            num_train_epochs=epochs,
            per_device_train_batch_size=self.batch_size,
            gradient_accumulation_steps=1,
            eval_strategy="steps",
            save_steps=evaluation_steps,
            logging_dir='./logs',
            weight_decay=weight_decay,
            learning_rate=2e-5,
            lr_scheduler_type=scheduler,
            warmup_steps=warmup_steps,
            logging_steps=500,
            save_total_limit=1,
            max_grad_norm=max_grad_norm
        )

        # Create a Trainer with custom callbacks and features
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=None,  # Add validation dataset if available
            tokenizer=self.tokenizer,
            compute_metrics=evaluator,
            callbacks=[callback] if callback else None,
            optimizers=(optimizer_class, optimizer_params),
            data_collator=None,
            #lr_scheduler_type=scheduler,
            # Pass the custom loss function and other parameters
            compute_loss_func=loss_fct if loss_fct else nn.CrossEntropyLoss(),
            max_grad_norm=max_grad_norm
        )

        # Wrap the model with accelerator for multi-device (e.g., multi-GPU)
        trainer.model, trainer.optimizer, trainer.data_collator = self.accelerator.prepare(
            trainer.model, trainer.optimizer, trainer.data_collator
        )

        self.accelerator.print("Starting training...")
        trainer.train()

    def predict(
        self,
        dataset,
        batch_size: int = 32,
        show_progress_bar=None,
        convert_to_numpy: bool = True,
        convert_to_tensor: bool = False,
        output_prob: bool = True
    ):
        # Use the accelerator to prepare the model for inference
        dataloader = self.create_batch(dataset, batch_size=batch_size, shuffle=False)
        model = self.accelerator.prepare(self.model)

        model.eval()
        pred_scores = []

        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Iteration", disable=not show_progress_bar):
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    return_dict=True
                )
                logits = outputs.logits

                if output_prob:
                    logits = torch.sigmoid(logits) if self.num_labels == 1 else F.softmax(logits, dim=1)

                pred_scores.extend(self.accelerator.gather(logits).cpu())

        pred_scores = torch.stack(pred_scores)

        if convert_to_tensor:
            return pred_scores
        if convert_to_numpy:
            return pred_scores.numpy()
        return pred_scores.tolist()

    def save(self, path: str, *, safe_serialization: bool = True, **kwargs) -> None:
        self.accelerator.print(f"Save model to {path}")
        self.accelerator.wait_for_everyone()
        if self.accelerator.is_main_process:
            self.model.save_pretrained(path, safe_serialization=safe_serialization, **kwargs)
            self.tokenizer.save_pretrained(path, **kwargs)
'''

'\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments\nfrom accelerate import Accelerator\nimport torch\nimport os\nimport time\nfrom tqdm import tqdm, trange\nfrom typing import Callable\nfrom torch import nn\nimport torch.nn.functional as F\nfrom torch.utils.data import DataLoader\n\nclass CrossEncoderForMultiClass:\n    def __init__(\n        self,\n        model_name: str,\n        num_labels: int = None,\n        batch_size: int = 32,\n        output_path: str = None\n    ):\n        # Initialize the accelerator\n        self.accelerator = Accelerator()\n\n        # Initialize the tokenizer and model\n        self.tokenizer = AutoTokenizer.from_pretrained(model_name)\n        ignore_mismatched_sizes = num_labels == 1\n\n        self.model = AutoModelForSequenceClassification.from_pretrained(\n            model_name,\n            num_labels=num_labels,\n            ignore_mismatched_sizes=ignore_mismatched_sizes\n        )\n      

In [13]:
#import csv
import logging
import os
from contextlib import nullcontext
from typing import TYPE_CHECKING
import json
import wandb
from sklearn.preprocessing import LabelBinarizer
from sentence_transformers.evaluation.SentenceEvaluator import SentenceEvaluator
from sentence_transformers import SentenceTransformer

if TYPE_CHECKING:
    from sentence_transformers.SentenceTransformer import SentenceTransformer

logger = logging.getLogger(__name__)


class CrossEncoderEvaluator(SentenceEvaluator):

    def __init__(
        self,
        val_dataloader,
        vali_data,
        metrics_save_path,
        name: str = "",
        batch_size: int = 32,
        show_progress_bar: bool = False,
        loss_fn = nn.CrossEntropyLoss()
        #write_csv: bool = True,
        #truncate_dim: int | None = None,
    ):
        self.val_dataloader= val_dataloader
        self.vali_data = vali_data
        self.name = name
        self.batch_size = batch_size

        self.show_progress_bar = show_progress_bar
        self.loss_fn = loss_fn
        self.json_file = metrics_save_path

    def append_to_json(self, file_path, new_dict, overwrite = False):
        # Check if the file exists
        if overwrite == False:
            if os.path.exists(file_path):
                # Open and load existing JSON data
                with open(file_path, 'r') as file:
                    data = json.load(file)

                # Ensure data is a list
                if not isinstance(data, list):
                    raise ValueError("JSON file must contain a list of dictionaries.")
            else:
                # If file doesn't exist, start with an empty list
                data = []

            # Append the new dictionary
            data.append(new_dict)
        else:
            data = new_dict

        # Ensure the directory exists
        folder_path = os.path.dirname(file_path)
        if folder_path:  # Check if there is a folder path in file_path
            os.makedirs(folder_path, exist_ok=True)
        # Save the updated data back to the file
        with open(file_path, 'w') as file:
            json.dump(data, file)

    def __call__(
        self,
        model: SentenceTransformer,
        train_loss,
        time,
        epoch: int = -1,
        steps: int = -1
    ) -> dict[str, float]:

        if epoch != -1:
            if steps == -1:
                out_txt = f" after epoch {epoch}"
            else:
                out_txt = f" in epoch {epoch} after {steps} steps"
        else:
            out_txt = ""
        #if self.truncate_dim is not None:
        #    out_txt += f" (truncated to {self.truncate_dim})"

        scores = self.compute_metrices(model)
        scores["train_loss"] = train_loss

        print(f"Epoch: {epoch}, step: {steps}, time: {time}, scores: ", scores)
        scores["epoch"] = epoch
        scores["steps"] = steps

        scores["train_time"] = time

        wandb.log(scores)
        self.append_to_json(self.json_file, scores)

        return scores

    def compute_metrices(self, model):
        best_f1 = -0.01
        best_threshold = 0.50
        scores = model.predict(self.val_dataloader, output_prob = False, batch_size = self.batch_size)
        for threshold in np.arange(0.01, 1.0, 0.01):
            pred = (np.array(scores) > threshold).astype(int)
            self.vali_data["pred"] = pred
            #grouped_scores = self.vali_data.groupby(["stance", "contributor_id", "chat_group_id"]).agg({"pred": "max", "label": "max"}).reset_index()
            #grouped_scores["pred"]= (np.array(grouped_scores["pred"]) > 1).astype(int)
            f1 = f1_score(self.vali_data["label"], self.vali_data["pred"])
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

        pred = (np.array(scores) > best_threshold).astype(int)
        self.vali_data["pred"] = pred
        #grouped_scores = self.vali_data.groupby(["stance", "contributor_id", "chat_group_id"]).agg({"pred": "max", "label": "max"}).reset_index()
        #grouped_scores["pred"]= (np.array(grouped_scores["pred"]) > 1).astype(int)
        y_true = self.vali_data["label"].values
        y_pred = self.vali_data["pred"].values

        output_scores = {}
        #display(y_true, y_scores, y_pred, scores)

        # Calculate metrics with 'weighted' average to account for class imbalance
        f1 = f1_score(y_true, y_pred, zero_division=1)
        precision = precision_score(y_true, y_pred, zero_division=1)
        recall = recall_score(y_true, y_pred, zero_division=1)

        output_scores = {
            "vali_f1": f1,
            #"vali_auc": auc_score,
            "vali_precision": precision,
            "vali_recall": recall,
            #"vali_loss": loss.item(),
            "weighted_precision":  precision_score(y_true, y_pred, zero_division=1, average = "weighted"),
            "weighted_recall":  recall_score(y_true, y_pred, zero_division=1, average = "weighted"),
            "weighted_f1":  f1_score(y_true, y_pred, zero_division=1, average = "weighted"),
            "macro_precision":  precision_score(y_true, y_pred, zero_division=1, average = "macro"),
            "macro_recall":  recall_score(y_true, y_pred, zero_division=1, average = "macro"),
            "macro_f1":  f1_score(y_true, y_pred, zero_division=1, average = "macro")
        }

        return output_scores

In [14]:
import torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print(torch.cuda.is_available())  # Should return True
print(torch.cuda.current_device())  # Check current device ID
print(torch.cuda.get_device_name(0))  # Should return your GPU model

True
0
NVIDIA A100-SXM4-40GB


In [15]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [16]:
import os
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels = 1,
            ignore_mismatched_sizes=True)

for name, param in model.named_parameters():
    print(name, param.shape, param.requires_grad)
model.to(device)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at Jsevisal/CrossEncoder-ModernBERT-base-qnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.embeddings.tok_embeddings.weight torch.Size([50368, 768]) True
model.embeddings.norm.weight torch.Size([768]) True
model.layers.0.attn.Wqkv.weight torch.Size([2304, 768]) True
model.layers.0.attn.Wo.weight torch.Size([768, 768]) True
model.layers.0.mlp_norm.weight torch.Size([768]) True
model.layers.0.mlp.Wi.weight torch.Size([2304, 768]) True
model.layers.0.mlp.Wo.weight torch.Size([768, 1152]) True
model.layers.1.attn_norm.weight torch.Size([768]) True
model.layers.1.attn.Wqkv.weight torch.Size([2304, 768]) True
model.layers.1.attn.Wo.weight torch.Size([768, 768]) True
model.layers.1.mlp_norm.weight torch.Size([768]) True
model.layers.1.mlp.Wi.weight torch.Size([2304, 768]) True
model.layers.1.mlp.Wo.weight torch.Size([768, 1152]) True
model.layers.2.attn_norm.weight torch.Size([768]) True
model.layers.2.attn.Wqkv.weight torch.Size([2304, 768]) True
model.layers.2.attn.Wo.weight torch.Size([768, 768]) True
model.layers.2.mlp_norm.weight torch.Size([768]) True
model.layers.2.mlp

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [17]:
from datasets import DatasetDict

# Assuming `train`, `test`, and `vali` are already defined as Dataset objects
data = DatasetDict({
    "train": train_dataloader,
    "test": test_dataloader,
    "validation": vali_dataloader
})

# Access the train dataset
print(data["train"])

# View the split dataset sizes
print("Train size:", len(data['train']))
print("Validation size:", len(data['validation']))
print("Test size:", len(data['test']))

Dataset({
    features: ['input_ids', 'attention_mask', 'label'],
    num_rows: 7348
})
Train size: 7348
Validation size: 674
Test size: 1349


In [18]:
version = "v6"
folder = f"/content/classification/{model_name}_{version}"
model_save_path = f"{folder}/model/"
metrics_save_path = f"{folder}/metrics/"
num_labels = 1
epochs = 1
lr = 0.0000001
train_batch_size = 16
eval_batch_size = 8
warmup_steps = 1000
weight_decay = 0
evaluation_steps = 100 # How many steps before you eval
optimizer_class = "adam"
scheduler = 'warmupcosinewithhardrestarts' # constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts


trainer = CrossEncoderTrainer(
    project_name = "colab_test",
    model_name = model_name,
    num_labels = num_labels,
    epochs = epochs,
    lr=lr,
    train_batch_size = train_batch_size,
    eval_batch_size = eval_batch_size,
    warmup_steps = warmup_steps,
    weight_decay = weight_decay,
    evaluation_steps = evaluation_steps,
    optimizer_class = optimizer_class,
    model_save_path = model_save_path,
    metrics_save_path=metrics_save_path,
    scheduler = scheduler,
    version = version
)

trainer.train(
    data["train"],
    data["validation"],
    vali_data
)




Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at Jsevisal/CrossEncoder-ModernBERT-base-qnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wilson_ng (wilson_ng-govtech). Use `wandb login --relogin` to force relogin


Training model
Starting model training...


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/7348 [00:00<?, ?it/s]

W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125] WON'T CONVERT compiled_embeddings /usr/local/lib/python3.10/dist-packages/transformers/models/modernbert/modeling_modernbert.py line 212 
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125] due to: 
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125] Traceback (most recent call last):
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/output_graph.py", line 1446, in _call_user_compiler
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/repro/after_dynamo.py", line 129, in __call__
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py:1125]     compiled_gm = compiler_fn(gm, example_inputs)
W0511 06:17:34.836000 94 torch/_dynamo/convert_frame.py

Epoch: 0, step: 100, time: 1.555772304534912, scores:  {'vali_f1': 0.5828437132784959, 'vali_precision': 0.4834307992202729, 'vali_recall': 0.7337278106508875, 'weighted_precision': 0.462275256943517, 'weighted_recall': 0.4732937685459941, 'weighted_f1': 0.4347198443444089, 'macro_precision': 0.46221229402007435, 'macro_recall': 0.4725186672302057, 'macro_f1': 0.4342789994963908, 'train_loss': 0.6859716203808784}
Save model to /content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/
Save model to /content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/
Epoch: 0, step: 200, time: 1.5695741176605225, scores:  {'vali_f1': 0.5828437132784959, 'vali_precision': 0.4834307992202729, 'vali_recall': 0.7337278106508875, 'weighted_precision': 0.462275256943517, 'weighted_recall': 0.4732937685459941, 'weighted_f1': 0.4347198443444089, 'macro_precision': 0.46221229402007435, 'macro_recall': 0.4725186672302057, 'macro_f1': 0.4342789994963908, 'train_loss': 0.6

In [19]:
!gsutil -m cp -r /content/classification/Jsevisal gs://mddi-reach-conversation/modernbert-output/

Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/metrics/classification_evaluation_Jsevisal_CrossEncoder-ModernBERT-base-qnli_v6_results.json [Content-Type=application/json]...
Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/special_tokens_map.json [Content-Type=application/json]...
Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/config.json [Content-Type=application/json]...
Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/tokenizer.json [Content-Type=application/json]...
Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/tokenizer_config.json [Content-Type=application/json]...
Copying file:///content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v6/model/model.safetensors [Content-Type=application/octet-stream]...
Copying file:///content/classification/Jsevisal/CrossEncod

# Test

In [1]:
import os
import torch
from transformers import AutoModelForSequenceClassification
model_name ="Jsevisal/CrossEncoder-ModernBERT-base-qnli"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folder = f"/content/classification/Jsevisal/CrossEncoder-ModernBERT-base-qnli_v2/model"

model = AutoModelForSequenceClassification.from_pretrained(folder, local_files_only=True)

for name, param in model.named_parameters():
    print(name, param.shape, param.requires_grad)
model.to(device)

[2025-05-11 07:46:11,597] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
model.embeddings.tok_embeddings.weight torch.Size([50368, 768]) True
model.embeddings.norm.weight torch.Size([768]) True
model.layers.0.attn.Wqkv.weight torch.Size([2304, 768]) True
model.layers.0.attn.Wo.weight torch.Size([768, 768]) True
model.layers.0.mlp_norm.weight torch.Size([768]) True
model.layers.0.mlp.Wi.weight torch.Size([2304, 768]) True
model.layers.0.mlp.Wo.weight torch.Size([768, 1152]) True
model.layers.1.attn_norm.weight torch.Size([768]) True
model.layers.1.attn.Wqkv.weight torch.Size([2304, 768]) True
model.layers.1.attn.Wo.weight torch.Size([768, 768]) True
model.layers.1.mlp_norm.weight torch.Size([768]) True
model.layers.1.mlp.Wi.weight torch.Size([2304, 768]) True
model.layers.1.mlp.Wo.weight torch.Size([768, 1152]) True
model.layers.2.attn_norm.weight torch.Size([768]) True
model.layers.2.attn.Wqkv.weight torch.Size([2304, 768]) True
model.laye

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [3]:
version = "v6"
model_version_name = model_name.replace("/", "_") + "_" + version

def create_batch(dataset, batch_size=None, shuffle=True, stratified=True):


    # If shuffle is True, shuffle the dataset first
    if shuffle:
        dataset = dataset.shuffle(seed=42)  # Shuffle with a seed for reproducibility

    # Set the format for Hugging Face datasets (before passing to DataLoader)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    if stratified:
        # Get the labels and calculate the class distribution
        labels = np.array(dataset['label'])  # Convert labels to numpy array
        class_counts = torch.tensor([sum(labels == i) for i in set(labels)])  # Count each class
        class_weights = 1.0 / class_counts  # Calculate inverse frequency (more weight for less frequent classes)
        sample_weights = class_weights[labels]  # Assign a weight to each sample based on its label

        # Create a WeightedRandomSampler for stratified sampling
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

        # Create the DataLoader using the sampler
        dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    else:
        # Without stratified sampling
        dataloader = DataLoader(dataset, batch_size=batch_size)

    return dataloader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True


In [17]:
from torch.utils.data import DataLoader, WeightedRandomSampler
from tqdm import tqdm

test_batchloader = create_batch(test_dataloader, batch_size=1, shuffle=False, stratified=False)

label = []
test_pred_scores = []
for batch in tqdm(
    test_batchloader, desc="Iteration", smoothing=0.05
):
    model.eval()
    features = {k: torch.Tensor(v).to(device) for k, v in batch.items() if k != 'label'}
    model_predictions = model(**features, return_dict=True)
    label += batch["label"].tolist()
    logits = torch.sigmoid(model_predictions.logits).detach().cpu().numpy().tolist()
    test_pred_scores.extend(logits)



Iteration:   0%|          | 0/1349 [00:00<?, ?it/s]W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125] WON'T CONVERT compiled_embeddings /usr/local/lib/python3.10/dist-packages/transformers/models/modernbert/modeling_modernbert.py line 212 
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125] due to: 
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125] Traceback (most recent call last):
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/output_graph.py", line 1446, in _call_user_compiler
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/repro/after_dynamo.py", line 129, in __call__
W0511 07:51:09.152000 18446 torch/_dynamo/convert_frame.py:1125]     compiled_gm = compiler_fn(gm,

NameError: name 'np' is not defined

In [21]:
import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

prediction = [pred[0] for pred in test_pred_scores]
test_data["score"] = prediction

best_f1 = 0
best_threshold = 0

for threshold in np.arange(0.01, 1.0, 0.01):
    test_data["pred"] = (np.array(prediction) > threshold).astype(int)

    score = f1_score(test_data["label"], test_data["pred"])
    if score > best_f1:
        best_f1 = score
        best_threshold = threshold


In [22]:

test_data["pred"] = (np.array(test_data["score"] ) > best_threshold).astype(int)


def compute_metrics(true_labels, prediction):
    metrics = {}
    metrics["acuracy"] = accuracy_score(true_labels, prediction)
    metrics["f1"] = f1_score(true_labels, prediction)
    metrics["precision"] = precision_score(true_labels, prediction)
    metrics["recall"] = recall_score(true_labels, prediction)
    metrics["f1_weighted"] = f1_score(true_labels, prediction, average = "weighted")
    metrics["recall_weighted"] = recall_score(true_labels,prediction, average = "weighted")
    metrics["precision_weighted"] = precision_score(true_labels, prediction, average = "weighted")
    metrics["f1_marco"] = f1_score(true_labels, prediction, average = "macro")
    metrics["precision_marco"] = precision_score(true_labels, prediction, average = "macro")
    metrics["recall_marco"] = recall_score(true_labels, prediction, average = "macro")

    return metrics

compute_metrics(test_data["label"], test_data["pred"])

{'acuracy': 0.508524833209785,
 'f1': 0.6696562032884903,
 'precision': 0.5048835462058603,
 'recall': 0.9940828402366864,
 'f1_weighted': 0.35578815100782857,
 'recall_weighted': 0.508524833209785,
 'precision_weighted': 0.6410272214081587,
 'f1_marco': 0.35508859368476614,
 'precision_marco': 0.6413306619918191,
 'recall_marco': 0.507442608825624}